## Data cleaning, feature engineering, and modeling

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv('data/raw/voter-survey-december16.csv')

In [ ]:
pattern = '(imiss_).{1}(_2016)'
def return_issue_cols(df):
    issue_cols = []
    for col in df.columns:
        if re.search(pattern, col):
            issue_cols.append(col)
    return issue_cols

issue_col_list = return_issue_cols(df)

In [ ]:
# target column list  
target_col_list = ['presvote16post_2016']
df_target = df[target_col_list]

In [ ]:
def fill_nans(df):
    df.fillna(0)
    return df

In [ ]:
def to_str_dummies(df):
    df = df.astype(str)
    df_dummies = pd.get_dummies(df)
    return df_dummies

In [ ]:
df_dummies.corr()['presvote16post_2016'].abs().sort_values(ascending=False)

In [ ]:
df_predictors = df_dummies.drop('presvote16post_2016', axis=1)

## K means clustering

In [ ]:
X = df_predictors

In [ ]:
k_means = KMeans(n_clusters=2)
k_means.fit(X)

In [ ]:
labels = k_means.labels_

In [ ]:
y_pred = k_means.predict(X)

In [ ]:
plt.figure(figsize=(5,5))
cl_centers = k_means.cluster_centers_
plt.scatter(cl_centers[:, 0], cl_centers[:, 1], c='black', s=300)

In [ ]:
metrics.silhouette_score(X, labels, metric='euclidean')

In [ ]:
metrics.calinski_harabasz_score(X, labels)

In [ ]:
df_target = df[target_col_list]
print(df_target.shape)
df_target_arr = np.ravel(df_target)

In [ ]:
df_target_arr
y_pred.shape

In [ ]:
dataset = pd.DataFrame({ 'y_pred': y_pred}, 
                        columns=[ 'y_pred']
                        )

In [ ]:
dataset.head()

In [ ]:
target_pred_df = df_predictors.join(df_target).join(dataset)
target_pred_df.head(10)

In [ ]:
count = 0
for row in target_pred_df.itertuples():
    if (row.presvote16post_2016 == 2): #and (row.y_pred == 0):
        count +=1
print(count)


In [ ]:
2578/3545

In [ ]:
2795/3479

## Logistic regression

In [ ]:
X = df_predictors
y = df_target
X.head()

In [ ]:
X.isna().sum().any()
y.isna().sum()

In [ ]:
y = y.fillna(0)
y = np.ravel(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size = 0.2, 
                                                    stratify=y)

In [ ]:
lr = LogisticRegression(C=2, penalty='l2', 
                        solver='lbfgs', 
                        multi_class='auto', 
                        max_iter=1000)
lr.fit(X_train, y_train)
y_pred_test = lr.predict(X_test)

In [ ]:
df_target.columns

In [ ]:
cm = confusion_matrix(y_test, y_pred_test)
conf_matrix = pd.DataFrame(cm, index=['None','Clinton','Trump','Johnson','Stein','McMullin','Other','Did not vote'], 
                           columns=['Pred None','Pred Clinton','Pred Trump','Pred Johnson','Pred Stein','Pred McMullin','Pred Other','Pred Did Not Vote'])


conf_matrix

In [ ]:
# correct Clinton
correct = cm[1][1]
total = sum(cm[1])
pred_correct = correct/total
pred_correct

In [ ]:
# correct Trump
correct = cm[2][2]
total = sum(cm[2])
pred_correct = correct/total
pred_correct

In [ ]:
len(lr.coef_[1])

In [ ]:
coef_dict = dict(zip(X.columns, lr.coef_[0]))
coef_dict

## Feature importance

In [ ]:
predictors = X_train
selector = RFE(lr, n_features_to_select=1)
selector = selector.fit(predictors, y_train)

In [ ]:
len(X_train.columns)

In [ ]:
order = selector.ranking_
len(order)
order

In [ ]:
df_predictors.columns

In [ ]:
for i in df_predictors.index:
    rank = np.array(order)
    feat = np.array(df_predictors.columns)
    rank_feat = pd.DataFrame(rank,feat, columns=['rank'])

In [ ]:
rank_feat.sort_values(by='rank').tail(10)

## Cross-validation

In [ ]:
y = y.fillna(0)

In [ ]:
X.isna().sum().any()

In [ ]:
y = np.ravel(y)
y.shape

In [ ]:
scores = cross_val_score(lr, X, y, cv=5)

In [ ]:
scores

## Random forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_test = rf.predict(X_test)
score = accuracy_score(y_test, y_pred_test)

In [ ]:
score

In [ ]:
scores = cross_val_score(rf, X, y, cv=5)

In [ ]:
scores

In [ ]:
knn = KNeighborsClassifier()
scores = cross_val_score(knn, X, y, cv=5)
scores

In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred_test = svm.predict(X_test)
score = accuracy_score(y_test, y_pred_test)
score

In [ ]:
scores = cross_val_score(svm, X, y, cv=5)
scores

In [ ]:
svm = SVC(kernel='linear', class_weight='balanced')
svm.fit(X_train, y_train)
y_pred_test = svm.predict(X_test)
score = accuracy_score(y_test, y_pred_test)
score